# XGBoost Model Training and Tuning

In [1]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# get sagemaker-execution role
# https://github.com/aws/sagemaker-python-sdk/issues/300
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html
def resolve_sm_role():
    client = boto3.client('iam', region_name=boto3.Session().region_name)
    response_roles = client.list_roles(
        PathPrefix='/',
        MaxItems=123
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
            return role['Arn']
    raise Exception('need to create sagemaker execution role from aws console first')
    return

try:
    execution_role_ARN = sagemaker.get_execution_role()
except ValueError:
    execution_role_ARN = resolve_sm_role()

execution_role_ARN

'arn:aws:iam::339713004220:role/SageMakerRole'

In [3]:
# Store the current SageMaker session
session = sagemaker.Session()
# Define the global bucket name
bucket = "data14group1-ml"

### Prepare the container and parameters for model training

In [4]:
# Retrieve the location of the container which is provided by Amazon for using XGBoost.
# As a matter of convenience, the training and inference code both use the same container.
region = session.boto_region_name
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")

region

'ap-southeast-2'

In [5]:
# Initialize hyperparameters
hyperparameters = {
        "objective":'binary:logistic',
        "max_depth":"5",
        "eta":"0.1",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.8",
        "early_stopping_rounds":"10",
        "num_round":"500"}

# set an output path where the trained model will be saved
# bucket = sagemaker.Session().default_bucket()
prefix = 'xgboost'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.

# construct a SageMaker estimator that calls the xgboost-container
xgb = sagemaker.estimator.Estimator(image_uri=xgboost_container,
                                      hyperparameters=hyperparameters,
                                      role=execution_role_ARN,
                                      instance_count=1, 
                                      instance_type='ml.m5.xlarge', 
                                      volume_size=5, # 5 GB 
                                      output_path=output_path,
                                      sagemaker_session=session
                                   )

print("xgb object specified!")

xgb object specified!


### Fit the xgb estimator

In [6]:
# The label column is assumed to be the first column

# define the data type and paths to the training and validation datasets
content_type = "application/x-parquet"
prefix = "data"

train_input = TrainingInput(
                "s3://{}/{}/{}/".format(bucket, prefix, "train"),
                content_type=content_type
)
validation_input = TrainingInput(
                "s3://{}/{}/{}/".format(bucket, prefix, "validation"),
                content_type=content_type
)

# execute the XGBoost training job
xgb.fit({'train': train_input, 'validation': validation_input})

print('xgb estimator training complete')

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-12-05-55-11-276


2024-07-12 05:55:11 Starting - Starting the training job...
2024-07-12 05:55:28 Starting - Preparing the instances for training...
2024-07-12 05:55:47 Downloading - Downloading input data...
2024-07-12 05:56:17 Downloading - Downloading the training image...
2024-07-12 05:57:03 Training - Training image download completed. Training in progress...[2024-07-12 05:57:14.735 ip-10-0-217-167.ap-southeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-07-12 05:57:14.757 ip-10-0-217-167.ap-southeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-07-12:05:57:15:INFO] Imported framework sagemaker_xgboost_container.training
[2024-07-12:05:57:15:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-07-12:05:57:15:INFO] No GPUs detected (normal if no gpus installed)
[2024-07-12:05:57:15:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-07-12:05:57:15:WARNING] 

### Load a trained model from S3 bucket

In [ ]:
from sagemaker.model import Model

model_name = ""
# Define the S3 path to the model
model_url = f"s3://{bucket}/xgboost/output/{model_name}/output/model.tar.gz"

# Create a SageMaker model
xgb = Model(
    model_data=model_url,
    image_uri=xgboost_container,
    role=execution_role_ARN
)

### Create the hyperparameter tuner (after model being trained)

Now that the base estimator has been set up we need to construct a hyperparameter tuner object which we will use to request SageMaker construct a hyperparameter tuning job.

**Note:** If you don't want the hyperparameter tuning job to take too long, make sure to not set the total number of models (jobs) too high.

In [7]:
# Import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
                        'max_depth': IntegerParameter(3, 12),
                        'eta'      : ContinuousParameter(0.05, 0.5),
                        'min_child_weight': IntegerParameter(2, 8),
                        'subsample': ContinuousParameter(0.5, 0.9),
                        'gamma': ContinuousParameter(0, 10)}

# create the tuner object:
xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb, # The estimator object to use as the basis for the training jobs.
                                               objective_metric_name = 'validation:rmse', # The metric used to compare trained models.
                                               objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                                               max_jobs = 4, # The total number of models to train
                                               max_parallel_jobs = 2, # The number of models to train in parallel
                                               hyperparameter_ranges = hyperparameter_ranges)

print("xgb tunner object specified!")

xgb tunner object specified!


### Fit the hyperparameter tuner

Now that the hyperparameter tuner object has been constructed, it is time to fit the various models and find the best performing model.

In [8]:
# The label column is assumed to be the first column

# define the data type and paths to the training and validation datasets
content_type = "application/x-parquet"
prefix = "data"

train_input = TrainingInput(
                "s3://{}/{}/{}/".format(bucket, prefix, "train"),
                content_type=content_type)
validation_input = TrainingInput(
                "s3://{}/{}/{}/".format(bucket, prefix, "validation"),
                content_type=content_type)

# execute the XGBoost training job
xgb_hyperparameter_tuner.fit({'train': train_input, 'validation': validation_input})

print('xgb estimator tunning complete')

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-240712-0632


.........................................................................................!
xgb estimator tunning complete


### Putting our model to work 
Now, the model is trained and tunned. So we can attach and deploy the best trained job to an endpoint

In [9]:
# attach the model:
xgb_attached = sagemaker.estimator.Estimator.attach(xgb_hyperparameter_tuner.best_training_job())


2024-07-12 06:37:12 Starting - Preparing the instances for training
2024-07-12 06:37:12 Downloading - Downloading the training image
2024-07-12 06:37:12 Training - Training image download completed. Training in progress.
2024-07-12 06:37:12 Uploading - Uploading generated training model
2024-07-12 06:37:12 Completed - Resource reused by training job: sagemaker-xgboost-240712-0632-004-4b9ccf1e


In [10]:
# now we deploy the model as an endpoint
xgb_predictor = xgb_attached.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')
print("\nxgb_predictor is ready")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-07-12-06-40-44-418
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-07-12-06-40-44-418
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-07-12-06-40-44-418


------!
xgb_predictor is ready


In [11]:
# create sagemaker-runtime object
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [13]:
endpoint_name = xgb_predictor.endpoint
endpoint_name

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-xgboost-2024-07-12-06-40-44-418'

In [14]:
# Try to invoke the endpoint and see if it works
# Body contains all the input features 
response = runtime.invoke_endpoint(EndpointName = endpoint_name, # The name of the endpoint we created
                                       ContentType = 'text/csv', # The data format that is expected
                                       Body = '1,6.67578125,3418,209,0.595703125,514,10.0,11,57,11,1599,0.1498791297340854,1.2884770346494763,0.22388993120700437,9.017543859649123,0.017543859649122806,46,0.02127659574468085')

In [15]:
# here is the response: the probablility of a user buy the product
response['Body'].read().decode('utf-8')

'0.44830912351608276\n'